In [7]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python312\\Scripts\\tabulate.exe' -> 'c:\\Python312\\Scripts\\tabulate.exe.deleteme'


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import pandas as pd

# 1. Datos de consumo por categoría
consumo = pd.read_csv("../data_final/dietas_consumo.csv")

# 2. Mapeo a grupos y recomendaciones AESAN (g/día)
grupo_map = {
    'Leche':'Lácteos','Yogurt':'Lácteos','Batidos':'Lácteos','Helado':'Lácteos','Queso fresco':'Lácteos','Mantequilla':'Lácteos',
    'Huevos':'Proteínas','Carne de res':'Proteínas','Embutidos':'Proteínas','Cerdo':'Proteínas','Cordero':'Proteínas','Pollo':'Proteínas',
    'Legumbres':'Legumbres','Frutos secos':'Frutos secos',
    'Arroz':'Cereales','Pasta':'Cereales','Pan':'Cereales','Cereales':'Cereales','Galletas':'Cereales',
    'Papas':'Tubérculos',
    'Tomate':'Verduras','Otros vegetales':'Verduras','Lechuga':'Verduras',
    'Manzana':'Frutas','Plátano':'Frutas','Cítricos':'Frutas','Durazno':'Frutas',
    'Aceitunas':'Aceites','Azúcar':'Azúcares',
    'Jugo':'Bebidas','Agua':'Bebidas','Gaseosa':'Bebidas','Café':'Bebidas','Cerveza':'Bebidas','Vino':'Bebidas'
}

recomend = {
    'Lácteos': 500, 'Proteínas': 100, 'Legumbres': 50, 'Frutos secos': 15,
    'Cereales': 300, 'Tubérculos': 200, 'Verduras': 400, 'Frutas': 300,
    'Aceites': 25, 'Azúcares': 30, 'Bebidas': 1500
}

# 3. Añadir columna de grupo
consumo['Grupo'] = consumo['Categoría'].map(grupo_map)

# 4. Sumar consumos por grupo
por_grupo = consumo.groupby('Grupo')[['DC', 'OLV', 'SUB', 'POB']].sum().reset_index()

# 5. Calcular porcentaje de adecuación
for diet in ['DC', 'OLV', 'SUB', 'POB']:
    por_grupo[f'%{diet}'] = (por_grupo[diet] / por_grupo['Grupo'].map(recomend) * 100).round(1)

# 6. Mostrar tabla resultante
tabla = por_grupo[['Grupo','%DC','%OLV','%SUB','%POB']]
print(tabla.to_markdown(index=False))



| Grupo        |   %DC |   %OLV |   %SUB |   %POB |
|:-------------|------:|-------:|-------:|-------:|
| Aceites      |  92   |   40   |  104   |    0   |
| Azúcares     |  20   |   33.3 |  140   |   33.3 |
| Bebidas      |  12.9 |   23.3 |   20.6 |   20.3 |
| Cereales     |  70   |  102   |  109   |  143.3 |
| Frutas       |  35.7 |   19.7 |   21.7 |   23.3 |
| Frutos secos |  33.3 |  120   |  233.3 |    0   |
| Legumbres    |  56   |  152   |   40   |  200   |
| Lácteos      |  50.8 |   68.2 |   52.8 |   20   |
| Proteínas    | 173   |   15   |  135   |   50   |
| Tubérculos   |  60   |    5   |   55   |  150   |
| Verduras     |  77.5 |   82.5 |   69   |   20   |


In [17]:
import pandas as pd

# 1. Cargar datos
consumo = pd.read_csv("../data_final/dietas_consumo.csv").rename(columns={"DC_ajustada":"DC"})
nutri   = pd.read_csv("../data_final/nutrientes_alimentos_agrupados_precios.csv")

# 2. Normalizar y filtrar categorías
consumo["Categoría"] = consumo["Categoría"].str.strip()
categorias = consumo["Categoría"].unique()
nutri = nutri[nutri["Alimento"].isin(categorias)]
nutri["Grupo"] = nutri["Grupo"].str.strip()

# 3. Nutrientes de interés
cols = {
    "Protein":"Proteínas",
    "Fiber, total dietary":"Fibra",
    "Potassium, K":"K",
    "Calcium, Ca":"Ca",
    "Iron, Fe":"Fe",
    "Magnesium, Mg":"Mg",
    "Vitamin A, RAE":"Vit A",
    "Vitamin C, total ascorbic acid":"Vit C",
    "Vitamin E, added":"Vit E",
    "Fatty acids, total saturated":"SAT",
    "Sodium, Na":"Na",
}

# 4. Composición media por grupo
comp_media = (
    nutri
    .groupby("Grupo")[list(cols.keys())]
    .mean()
    .rename(columns=cols)
)

# 5. Construir long
rows = []
for dieta in ["DC","OLV","SUB","POB"]:
    for _, r in consumo.iterrows():
        cat = r["Categoría"]
        gramos = r[dieta]
        if cat not in comp_media.index:
            continue
        comp = comp_media.loc[cat]
        for nutr in cols.values():
            ing = comp[nutr] * gramos/100
            rows.append({"Dieta": dieta, "Nutriente": nutr, "Ingesta": ing})

long = pd.DataFrame(rows)

# — Depuración —
print("Columnas de long:", long.columns.tolist())
print(long.head())

# 6. Agrupar usando los nombres correctos
sumas = long.groupby(["Dieta","Nutriente"])["Ingesta"].sum().unstack()

# 7. RDA y UL
rda = {"Proteínas":50,"Fibra":25,"K":3500,"Ca":1000,"Fe":18,"Mg":375,"Vit A":800,"Vit C":80,"Vit E":12}
ul  = {"SAT":20,"Na":2000,"Azúcar añadida":50}

# 8. Calcular TNR9 y TNL3
tnr9 = sumas[list(rda)].divide(pd.Series(rda), axis=1)*100
tnl3 = sumas[list(ul)].divide(pd.Series(ul), axis=1)*100
tnl3 = tnl3.clip(upper=100)

# 9. NRD9.3
nrd = pd.DataFrame({
    "TNR9": tnr9.sum(axis=1),
    "TNL3": tnl3.sum(axis=1)
})
nrd["NRD9.3"] = nrd["TNR9"] - nrd["TNL3"]

# 10. Salida
tabla = pd.concat([tnr9, tnl3, nrd], axis=1).round(1)
print(tabla.to_markdown())


Columnas de long: []
Empty DataFrame
Columns: []
Index: []


KeyError: 'Dieta'